In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pynapple as nap
from spatial_manifolds.toroidal import *
from spatial_manifolds.behaviour_plots import *
from matplotlib.colors import TwoSlopeNorm
from scipy.spatial import distance
from spatial_manifolds.circular_decoder import circular_decoder, cross_validate_decoder, cross_validate_decoder_time, circular_nanmean

from spatial_manifolds.data.curation import curate_clusters
from scipy.stats import zscore
from spatial_manifolds.util import gaussian_filter_nan
from spatial_manifolds.predictive_grid import compute_travel_projected, wrap_list
from spatial_manifolds.behaviour_plots import *
from spatial_manifolds.behaviour_plots import trial_cat_priority
from spatial_manifolds.detect_grids import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

fig_path = '/Users/harryclark/Documents/figs/FIGURE1/'


In [ ]:
# 'M29D22' throws errors I think because it has too few disengaged trials
# 'M26D14' throws an error, doesn't have field spacing column for the cell classification in of1
# 'M22D35' throws an error, doesn't have field spacing column for the cell classification in of1

sessions_to_use = ['M27D16', 'M29D22', 
                   'M21D18', 'M21D23', 'M21D24', 'M21D25', 'M21D26',
                   'M22D34', 'M22D36', 'M25D22', 'M25D23',
                   'M25D24', 'M25D25', 'M28D16', 'M28D17', 'M28D18',
                   'M28D20', 'M28D23', 'M28D25', 'M29D16',
                   'M29D17', 'M29D18', 'M29D19', 'M29D20', 'M29D21', 
                   'M29D23', 'M29D25', 'M20D14', 'M20D22', 'M29D22'
                   'M20D23', 'M20D25', 'M20D26', 'M21D17', 'M21D21',
                   'M21D22', 'M22D33', 'M22D37', 'M22D38', 'M22D39',
                   'M22D40', 'M22D41', 'M25D17', 'M25D19', 'M25D20',
                   'M26D11', 'M26D12', 'M26D13', 'M26D15',
                   'M26D16', 'M26D17', 'M26D19', 'M27D17',
                   'M27D18', 'M27D19', 'M27D20', 'M27D21', 'M27D24',
                   'M27D26', 'M28D19', 'M28D21', 'M20D15', 'M20D16',
                   'M20D17', 'M20D18', 'M20D19', 'M20D20', 'M20D21', 
                   'M20D24', 'M20D27', 'M20D28', 'M21D15', 'M21D16', 
                   'M21D19', 'M21D20', 'M21D27', 'M21D28', 'M25D16',
                   'M26D18', 'M27D23']

#sessions_to_use = ['M21D18', 'M21D23', 'M21D24', 'M21D25', 'M21D26',
#                   'M22D34', 'M22D35', 'M22D36', 'M25D22', 'M25D23',
#                   'M25D24', 'M25D25', 'M28D16', 'M28D17', 'M28D18']

df = pd.DataFrame()
for session in sessions_to_use:
    mouse = session.split('M')[-1].split('D')[0]
    day = session.split('M')[-1].split('D')[-1]

    # collect cell classifications
    gcs, ngs, ns, sc, ngs_ns, all = cell_classification_of1(mouse, day, percentile_threshold=99) # subset
    rc, rsc, vr_ns = cell_classification_vr(mouse, day)
    tcs, tcs_time, autocorrs, last_ephys_bin, beh, clusters = compute_vr_tcs(mouse, day)
    
    # get session info
    trial_groups, trial_colors = get_trial_groups_and_colors(beh, last_ephys_bin, tl, bs)
    sorted_trial_indices, sorted_trial_colors = get_sorted_trials_and_colors(beh, last_ephys_bin, tl, bs)
    tns = beh['trial_number']
    dt = beh['travel']-((tns[0]-1)*tl)
    n_bins = int(int(((np.ceil(np.nanmax(dt))//tl)+1)*tl)/bs)
    max_bound = int(((np.ceil(np.nanmax(dt))//tl)+1)*tl)
    min_bound = 0
    dt_bins = np.arange(0,max_bound,bs)
    x_true_dt = dt_bins[:last_ephys_bin]
    true_position = x_true_dt%tl
    trial_numbers = (x_true_dt//tl)+beh['trials']['number'][0]

    # loop over combinations
    for train_set, train_set_label in zip([np.array(['hit']), np.array(['try', 'run'])], ['eng', 'diseng']):
        for test_set, test_set_label in zip([np.array(['hit']), np.array(['try', 'run'])], ['eng', 'diseng']):
            for trial_type_set, trial_type_set_label in zip([np.array(['b','nb']), np.array(['b']), np.array(['nb'])], ['b_nb', 'b', 'nb']):
                for cluster_ids, cell_type_label in zip([gcs.cluster_id.values, ngs.cluster_id.values, vr_ns.cluster_id.values], ['GC', 'NGS', 'NS']):

                    tns_to_decode = np.array(beh['trials']['number'][(np.isin(beh['trials']['type'], trial_type_set)) &
                                                                    (np.isin(beh['trials']['performance'], test_set))])
                    tns_to_train = np.array(beh['trials']['number'][(np.isin(beh['trials']['type'], trial_type_set)) &
                                                                    (np.isin(beh['trials']['performance'], train_set))]) 
                    
                    tns_to_decode = tns_to_decode[tns_to_decode<=np.nanmax(trial_numbers)] # handles last ephys trials
                    tns_to_train = tns_to_train[tns_to_train<=np.nanmax(trial_numbers)] # handles last ephys trials
                    
                    print(f'n clusters: {len(cluster_ids)}')
                    print(f'n trials to decode: {len(tns_to_decode)}')
                    print(f'n trials to train {len(tns_to_train)}')

                    if len(cluster_ids)>10 and len(tns_to_decode) > 5 and len(tns_to_train) > 5:
                        tcs_ = {cluster_id: tcs[cluster_id] for cluster_id in cluster_ids if cluster_id in tcs}

                        predictions, errors = cross_validate_decoder(tcs_, true_position, trial_numbers, tns_to_decode, 
                                                                    tns_to_train, tl, bs, train=0.9, n=10, verbose=False)
                        
                        avg_predictions = circular_nanmean(predictions, tl, axis=2)
                        avg_errors = np.nanmean(errors, axis=2)
                        
                        tmp = pd.DataFrame()
                        tmp['train_set'] = [train_set_label]
                        tmp['test_set'] = [test_set_label]
                        tmp['cell_type'] = [cell_type_label]
                        tmp['trial_type'] = [trial_type_set_label]
                        tmp['error'] = [np.nanmean(avg_errors)]
                        tmp['error_over_location'] = [avg_errors.mean(axis=0)]
                        tmp['mouse'] = [mouse]
                        tmp['day'] = [day]
                        tmp['mouse_day'] = [session]
                        
                        df = pd.concat([df, tmp], ignore_index=True)


In [ ]:
df.to_pickle('/Users/harryclark/Downloads/test_train_linear_decoder.pkl')

In [ ]:
df = pd.read_pickle('/Users/harryclark/Downloads/test_train_linear_decoder.pkl')

In [ ]:
for cell_type_label in ['GC', 'NGS', 'NS']:
    for trial_type_set_label in ['b', 'nb']:
        ct_df = df[df['cell_type'] == cell_type_label]
        ct_df = ct_df[ct_df['trial_type'] == trial_type_set_label]

        # Calculate average error for each train_set and test_set combination
        avg_error = ct_df.groupby(['train_set', 'test_set'])['error'].mean().reset_index()
        eng_eng_error = avg_error[(avg_error['train_set'] == 'eng') & (avg_error['test_set'] == 'eng')].error.iloc[0]
        eng_diseng_error = avg_error[(avg_error['train_set'] == 'eng') & (avg_error['test_set'] == 'diseng')].error.iloc[0]
        diseng_eng_error = avg_error[(avg_error['train_set'] == 'diseng') & (avg_error['test_set'] == 'eng')].error.iloc[0]
        diseng_diseng_error = avg_error[(avg_error['train_set'] == 'diseng') & (avg_error['test_set'] == 'diseng')].error.iloc[0]

        # Create bar plot
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2, 2))
        ax.bar(x=[1,2,4,5], height=[eng_eng_error, 
                                    eng_diseng_error, 
                                    diseng_eng_error, 
                                    diseng_diseng_error], 
                                    color=[get_color_for_group(('rz1',trial_type_set_label,'hit')),
                                           get_color_for_group(('rz1',trial_type_set_label,'run')),
                                           get_color_for_group(('rz1',trial_type_set_label,'hit')),
                                           get_color_for_group(('rz1',trial_type_set_label,'run'))],
                                    edgecolor=[get_color_for_group(('rz1',trial_type_set_label,'hit')),
                                               get_color_for_group(('rz1',trial_type_set_label,'hit')),
                                               get_color_for_group(('rz1',trial_type_set_label,'run')),
                                               get_color_for_group(('rz1',trial_type_set_label,'run'))])

        # Overlay line plot for individual trajectories
        for mouse_day in ct_df['mouse_day'].unique():
            subset = ct_df[ct_df['mouse_day'] == mouse_day]

            eng_eng_error = subset[(subset['train_set'] == 'eng') & (subset['test_set'] == 'eng')].error
            eng_diseng_error = subset[(subset['train_set'] == 'eng') & (subset['test_set'] == 'diseng')].error
            diseng_eng_error = subset[(subset['train_set'] == 'diseng') & (subset['test_set'] == 'eng')].error
            diseng_diseng_error = subset[(subset['train_set'] == 'diseng') & (subset['test_set'] == 'diseng')].error

            if len(eng_eng_error)>0 and len(eng_diseng_error)>0 and len(diseng_eng_error)>0 and len(diseng_diseng_error)>0:
                ax.plot([1,2], [eng_eng_error.iloc[0], eng_diseng_error.iloc[0]], linestyle='-', color='grey', alpha=0.5)
                ax.plot([4,5], [diseng_eng_error.iloc[0], diseng_diseng_error.iloc[0]], linestyle='-', color='grey', alpha=0.5)

        ax.set_xticks([1,2,4,5])
        ax.set_xticklabels(['Eng', 'Dis', 'Eng', 'Dis'])
        ax.set_title(f'{cell_type_label}:{trial_type_set_label}')
        ax.set_ylim(0,60)
        ax.set_ylabel('Decoding error (cm)')
        plt.tight_layout()
        fig.savefig(f'{fig_path}/Decoding_vs_test_train_sets_{cell_type_label}_tt_{trial_type_set_label}.pdf', dpi=300, bbox_inches='tight')
        plt.show()



In [ ]:
def cdt2dna(pandas_series):
    new_array = []
    for i in range(len(pandas_series)):
        element = pandas_series.iloc[i]

        if len(np.shape(element)) == 0:
            new_array.append([element])
        else:
            new_array.append(element)

    return np.array(new_array)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

for cell_type_label in ['GC', 'NGS', 'NS']:
    for trial_type_set_label in ['b', 'nb']:
        ct_df = df[df['cell_type'] == cell_type_label]
        ct_df = ct_df[ct_df['trial_type'] == trial_type_set_label]
            
        # Calculate average error for each train_set and test_set combination
        eng_eng = ct_df[(ct_df['train_set'] == 'eng') & (ct_df['test_set'] == 'eng')]
        eng_diseng = ct_df[(ct_df['train_set'] == 'eng') & (ct_df['test_set'] == 'diseng')]
        

        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(2, 2))
        ax.axvspan(
            90,110,
            alpha=0.2,
            zorder=-10,
            edgecolor='none',
            facecolor='grey',
        )
        ax.plot(np.arange(bs/2, (tl+bs/2), bs), np.nanmean(cdt2dna(eng_eng['error_over_location']), axis=0), color= get_color_for_group(('rz1',trial_type_set_label,'hit')))
        ax.plot(np.arange(bs/2, (tl+bs/2), bs), np.nanmean(cdt2dna(eng_diseng['error_over_location']), axis=0), color= get_color_for_group(('rz1',trial_type_set_label,'run')))
        ax.fill_between(np.arange(bs/2, (tl+bs/2), bs), 
                        np.nanmean(cdt2dna(eng_eng['error_over_location']), axis=0) + stats.sem(cdt2dna(eng_eng['error_over_location']), nan_policy='omit'),
                        np.nanmean(cdt2dna(eng_eng['error_over_location']), axis=0) - stats.sem(cdt2dna(eng_eng['error_over_location']), nan_policy='omit'),
                        color= get_color_for_group(('rz1',trial_type_set_label,'hit')), alpha=0.3)
        ax.fill_between(np.arange(bs/2, (tl+bs/2), bs), 
                        np.nanmean(cdt2dna(eng_diseng['error_over_location']), axis=0) + stats.sem(cdt2dna(eng_diseng['error_over_location']), nan_policy='omit'),
                        np.nanmean(cdt2dna(eng_diseng['error_over_location']), axis=0) - stats.sem(cdt2dna(eng_diseng['error_over_location']), nan_policy='omit'),
                        color= get_color_for_group(('rz1',trial_type_set_label,'run')), alpha=0.3)

        ax.set_title(f'{cell_type_label}:{trial_type_set_label}')
        ax.set_xlim(0,tl)    
        ax.set_ylim(0,60)
        ax.set_xlabel('Position (cm)')
        ax.set_ylabel('Decoding error (cm)')
        plt.tight_layout()
        plt.show()
        fig.savefig(f'{fig_path}/Decoding_vs_location_test_train_sets_{cell_type_label}_tt_{trial_type_set_label}.pdf', dpi=300, bbox_inches='tight')



do some stats using mixed linear models to determine the significance of decoder performances in different cell populations 

In [ ]:
df

In [ ]:
import statsmodels.formula.api as smf

'''
Model Construction Notes:
1. Fixed effects: age, sex, reward order, task struct (0 = block, 1 = alt), context ID (all categorical), session (continuous). 
2. Added context & taststruct to agegroup interaction terms.
3. Random effect is animal ID in that sessions are grouped by belonging to the same animal.
4. Modeling random intercepts & slopes captures learning rate differences that I'm hoping to model.

'''

# Get LMM & Print Results
mtask = smf.mixedlm('ReqRate ~ 1 + Session + Session:C(AgeGroup) + C(AgeGroup) + Satiety + C(Sex):C(AgeGroup) + C(Sex) + C(RewardOrder) + C(ContextID) + C(ContextID):C(AgeGroup) + C(TaskStruct) + C(TaskStruct):C(AgeGroup)', groups = 'AnimalID', re_formula = '~1 + Session', data = df).fit(reml= True)
print(mtask.summary())
print(mtask.wald_test_terms(scalar = False))